In [1]:
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

In [2]:
sentence = "The wide road shimmered in the hot sun"
tokens = list(sentence.lower().split())
print(len(tokens))

8


In [3]:
vocab, index = {}, 1  # start indexing from 1
vocab['<pad>'] = 0  # add a padding token
for token in tokens:
  if token not in vocab:
    vocab[token] = index
    index += 1
vocab_size = len(vocab)
print(vocab)

{'<pad>': 0, 'the': 1, 'wide': 2, 'road': 3, 'shimmered': 4, 'in': 5, 'hot': 6, 'sun': 7}


In [4]:
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)

{0: '<pad>', 1: 'the', 2: 'wide', 3: 'road', 4: 'shimmered', 5: 'in', 6: 'hot', 7: 'sun'}


In [5]:
example_sequence = [vocab[word] for word in tokens]
print(example_sequence)

[1, 2, 3, 4, 5, 1, 6, 7]


In [6]:
window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(         #",_"means ignoring the last output
      example_sequence,
      vocabulary_size=vocab_size,
      window_size=window_size,
      negative_samples=0)

In [7]:
positive_skip_grams

[[1, 3],
 [2, 1],
 [3, 2],
 [6, 1],
 [2, 3],
 [5, 3],
 [1, 7],
 [7, 1],
 [6, 5],
 [5, 1],
 [3, 1],
 [4, 5],
 [3, 5],
 [5, 4],
 [1, 2],
 [1, 4],
 [7, 6],
 [3, 4],
 [6, 7],
 [1, 6],
 [1, 5],
 [4, 1],
 [4, 3],
 [5, 6],
 [4, 2],
 [2, 4]]

In [8]:
for target, context in positive_skip_grams:
  print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

(1, 3): (the, road)
(2, 1): (wide, the)
(3, 2): (road, wide)
(6, 1): (hot, the)
(2, 3): (wide, road)
(5, 3): (in, road)
(1, 7): (the, sun)
(7, 1): (sun, the)
(6, 5): (hot, in)
(5, 1): (in, the)
(3, 1): (road, the)
(4, 5): (shimmered, in)
(3, 5): (road, in)
(5, 4): (in, shimmered)
(1, 2): (the, wide)
(1, 4): (the, shimmered)
(7, 6): (sun, hot)
(3, 4): (road, shimmered)
(6, 7): (hot, sun)
(1, 6): (the, hot)
(1, 5): (the, in)
(4, 1): (shimmered, the)
(4, 3): (shimmered, road)
(5, 6): (in, hot)
(4, 2): (shimmered, wide)
(2, 4): (wide, shimmered)


In [9]:
_ #all postive skip grams, therefore the result will be all 1.

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [10]:
# Get target and context words of the first  positive skip-gram.
target_word, context_word = positive_skip_grams[0]

# Pass the context word as true class to exclude it from being sampled for negative sampling
context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))

In [11]:
context_class

<tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[3]])>

In [12]:
# Set the number of negative samples per positive context.
num_ns = 4
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # class that should be sampled as 'positive'
    num_true=1,  # each positive skip-gram has 1 positive context class
    num_sampled=num_ns,  # number of negative context words to sample
    unique=True,  # all the negative samples should be unique
    range_max=vocab_size,  # pick index of the samples from [0, vocab_size]
    seed = 42,  # seed for reproducibility
    name="negative_sampling"  # name of this operation
)

In [13]:
negative_sampling_candidates

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([2, 1, 4, 3])>

In [14]:
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

['wide', 'the', 'shimmered', 'road']


In [15]:
# Reduce a dimension so you can use concatenation (in the next step).
squeezed_context_class = tf.squeeze(context_class, 1)

# Concatenate a positive context word with negative sampled words.
context = tf.concat([squeezed_context_class, negative_sampling_candidates], 0)

context

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([3, 2, 1, 4, 3])>

In [16]:
# Label the first context word as `1` (positive) followed by `num_ns` `0`s (negative).
label = tf.constant([1] + [0]*num_ns, dtype="int64")

label

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([1, 0, 0, 0, 0])>

A tuple of (target_word, context, label) tensors constitutes one training example for training the skip-gram negative sampling word2vec model. Notice that the target is of shape (1,) while the context and label are of shape (1+num_ns,)

In [17]:
#You can use the tf.keras.preprocessing.sequence.make_sampling_table to generate a word-frequency rank based probabilistic sampling table and pass it to the skipgrams function. Inspect the sampling probabilities for a vocab_size of 10.
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)

[0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381 0.01347162 0.01474487 0.0159558 ]


sampling_table[i] denotes the probability of sampling the i-th most common word in a dataset. The function assumes a Zipf's distribution of the word frequencies for sampling.

**Generate training data**

Compile all the steps described above into a function that can be called on a list of vectorized sentences obtained from any text dataset. Notice that the sampling table is built before sampling skip-gram word pairs. You will use this function in the later sections.

In [18]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  target_words, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=seed,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      target_words.append(target_word)
      contexts.append(context)
      labels.append(label)

  return target_words, contexts, labels

In [19]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [20]:
with open(path_to_file) as f:
  lines = f.read().splitlines()
for line in lines[:20]:
  print(line)

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.


In [21]:
#remove
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

In [22]:
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
vocab_size = 4096
sequence_length = 10

# Use the `TextVectorization` layer to normalize, split, and map text strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

In [23]:
vectorize_layer.adapt(text_ds.batch(1024))

In [24]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'the', 'and', 'to', 'i', 'of', 'you', 'my', 'a', 'that', 'in', 'is', 'not', 'for', 'with', 'me', 'it', 'be', 'your']


In [26]:
# Vectorize the data in text_ds.
AUTOTUNE = tf.data.AUTOTUNE

text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [27]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

32777


In [28]:
for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']
[138  36 982 144 673 125  16 106   0   0] => ['before', 'we', 'proceed', 'any', 'further', 'hear', 'me', 'speak', '', '']
[34  0  0  0  0  0  0  0  0  0] => ['all', '', '', '', '', '', '', '', '', '']
[106 106   0   0   0   0   0   0   0   0] => ['speak', 'speak', '', '', '', '', '', '', '', '']
[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']


In [30]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=42)

targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

100%|██████████| 32777/32777 [00:24<00:00, 1361.05it/s]




targets.shape: (65114,)
contexts.shape: (65114, 5)
labels.shape: (65114, 5)


In [31]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<_BatchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


In [32]:
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<_PrefetchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


In [33]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim)

  def call(self, pair):
    target, context = pair

    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    word_emb = self.target_embedding(target)
    context_emb = self.context_embedding(context)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    return dots

In [34]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [35]:
word2vec.fit(dataset, epochs=20)

Epoch 1/20
63/63 [==============================] - 2s 22ms/step - loss: 1.6082 - accuracy: 0.2347
Epoch 2/20
63/63 [==============================] - 1s 17ms/step - loss: 1.5888 - accuracy: 0.5610
Epoch 3/20
63/63 [==============================] - 1s 16ms/step - loss: 1.5410 - accuracy: 0.6074
Epoch 4/20
63/63 [==============================] - 1s 17ms/step - loss: 1.4576 - accuracy: 0.5792
Epoch 5/20
63/63 [==============================] - 1s 16ms/step - loss: 1.3581 - accuracy: 0.5858
Epoch 6/20
63/63 [==============================] - 1s 16ms/step - loss: 1.2596 - accuracy: 0.6117
Epoch 7/20
63/63 [==============================] - 1s 16ms/step - loss: 1.1679 - accuracy: 0.6457
Epoch 8/20
63/63 [==============================] - 1s 23ms/step - loss: 1.0835 - accuracy: 0.6806
Epoch 9/20
63/63 [==============================] - 1s 17ms/step - loss: 1.0055 - accuracy: 0.7134
Epoch 10/20
63/63 [==============================] - 1s 16ms/step - loss: 0.9334 - accuracy: 0.7423
Epoch 11/

In [37]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [38]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [39]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>